<a href="https://colab.research.google.com/github/wr0124/Learning_essential/blob/main/CorssAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#source

"self_attention"
old: https://github.com/huggingface/diffusers_all/blob/main/src/diffusers/models/attention.py

new: https://github.com/huggingface/diffusers/blob/main/src/diffusers/models/attention_processor.py#L41

https://github.com/huggingface/diffusers/blob/main/src/diffusers/models/attention.py

branch: https://github.com/huggingface/diffusers/blob/4125756e88e82370c197fecf28e9f0b4d7eee6c3/src/diffusers/models/cross_attention.py

https://github.com/openai/consistency_models/blob/e32b69ee436d518377db86fb2127a3972d0d8716/cm/unet.py#L478

https://github.com/openai/guided-diffusion/blob/22e0df8183507e13a7813f8d38d51b072ca1e67c/guided_diffusion/unet.py#L361


other:
show attention: https://github.com/google/prompt-to-prompt/blob/main/prompt-to-prompt_ldm.ipynb

"cross_attention"

https://github.com/openai/point-e/blob/fc8a607c08a3ea804cc82bf1ef8628f88a3a5d2f/point_e/models/perceiver.py#L11

https://github.com/openai/shap-e/blob/50131012ee11c9d2617f3886c10f000d3c7a3b43/shap_e/models/generation/perceiver.py#L13

https://github.com/CompVis/latent-diffusion/blob/a506df5756472e2ebaf9078affdde2c4f1502cd4/ldm/modules/attention.py#L152

https://blog.csdn.net/weixin_54338498/article/details/135051918

"attention_map"
https://github.com/google/prompt-to-prompt/tree/main


# From animateddiff lib
juliew@neptune7:~/miniconda3/envs/animatediff/lib/python3.10/site-packages/diffusers/models

## ori

In [9]:
# feedforward
class GEGLU(nn.Module):
    r"""
    A variant of the gated linear unit activation function from https://arxiv.org/abs/2002.05202.

    Parameters:
        dim_in (`int`): The number of channels in the input.
        dim_out (`int`): The number of channels in the output.
    """

    def __init__(self, dim_in: int, dim_out: int):
        super().__init__()
        self.proj = nn.Linear(dim_in, dim_out * 2)

    def gelu(self, gate):
        if gate.device.type != "mps":
            return F.gelu(gate)
        # mps: gelu is not implemented for float16
        return F.gelu(gate.to(dtype=torch.float32)).to(dtype=gate.dtype)

    def forward(self, hidden_states):
        hidden_states, gate = self.proj(hidden_states).chunk(2, dim=-1)
        return hidden_states * self.gelu(gate)

class FeedForward(nn.Module):
    r"""
    A feed-forward layer.

    Parameters:
        dim (`int`): The number of channels in the input.
        dim_out (`int`, *optional*): The number of channels in the output. If not given, defaults to `dim`.
        mult (`int`, *optional*, defaults to 4): The multiplier to use for the hidden dimension.
        dropout (`float`, *optional*, defaults to 0.0): The dropout probability to use.
        activation_fn (`str`, *optional*, defaults to `"geglu"`): Activation function to be used in feed-forward.
    """

    def __init__(
        self,
        dim: int,
        dim_out: Optional[int] = None,
        mult: int = 4,
        dropout: float = 0.0,
        activation_fn: str = "geglu",
    ):
        super().__init__()
        inner_dim = int(dim * mult)
        dim_out = dim_out if dim_out is not None else dim

        if activation_fn == "gelu":
            act_fn = GELU(dim, inner_dim)
        elif activation_fn == "geglu":
            act_fn = GEGLU(dim, inner_dim)
        elif activation_fn == "geglu-approximate":
            act_fn = ApproximateGELU(dim, inner_dim)

        self.net = nn.ModuleList([])
        # project in
        self.net.append(act_fn)
        # project dropout
        self.net.append(nn.Dropout(dropout))
        # project out
        self.net.append(nn.Linear(inner_dim, dim_out))

    def forward(self, hidden_states):
        for module in self.net:
            hidden_states = module(hidden_states)
        return hidden_states




class CrossAttention(nn.Module):
    r"""
    A cross attention layer.

    Parameters:
        query_dim (`int`): The number of channels in the query.
        cross_attention_dim (`int`, *optional*):
            The number of channels in the encoder_hidden_states. If not given, defaults to `query_dim`.
        heads (`int`,  *optional*, defaults to 8): The number of heads to use for multi-head attention.
        dim_head (`int`,  *optional*, defaults to 64): The number of channels in each head.
        dropout (`float`, *optional*, defaults to 0.0): The dropout probability to use.
        bias (`bool`, *optional*, defaults to False):
            Set to `True` for the query, key, and value linear layers to contain a bias parameter.
    """

    def __init__(
        self,
        query_dim: int,
        cross_attention_dim: Optional[int] = None,
        heads: int = 8,
        dim_head: int = 64,
        dropout: float = 0.0,
        bias=False,
        upcast_attention: bool = False,
        upcast_softmax: bool = False,
        added_kv_proj_dim: Optional[int] = None,
        norm_num_groups: Optional[int] = None,
    ):
        super().__init__()
        inner_dim = dim_head * heads
        cross_attention_dim = cross_attention_dim if cross_attention_dim is not None else query_dim
        self.upcast_attention = upcast_attention
        self.upcast_softmax = upcast_softmax

        self.scale = dim_head**-0.5

        self.heads = heads
        # for slice_size > 0 the attention score computation
        # is split across the batch axis to save memory
        # You can set slice_size with `set_attention_slice`
        self.sliceable_head_dim = heads
        self._slice_size = None
        self._use_memory_efficient_attention_xformers = False
        self.added_kv_proj_dim = added_kv_proj_dim

        if norm_num_groups is not None:
            self.group_norm = nn.GroupNorm(num_channels=inner_dim, num_groups=norm_num_groups, eps=1e-5, affine=True)
        else:
            self.group_norm = None

        self.to_q = nn.Linear(query_dim, inner_dim, bias=bias)
        self.to_k = nn.Linear(cross_attention_dim, inner_dim, bias=bias)
        self.to_v = nn.Linear(cross_attention_dim, inner_dim, bias=bias)

        if self.added_kv_proj_dim is not None:
            self.add_k_proj = nn.Linear(added_kv_proj_dim, cross_attention_dim)
            self.add_v_proj = nn.Linear(added_kv_proj_dim, cross_attention_dim)

        self.to_out = nn.ModuleList([])
        self.to_out.append(nn.Linear(inner_dim, query_dim))
        self.to_out.append(nn.Dropout(dropout))

    def reshape_heads_to_batch_dim(self, tensor):
        batch_size, seq_len, dim = tensor.shape
        head_size = self.heads
        tensor = tensor.reshape(batch_size, seq_len, head_size, dim // head_size)
        tensor = tensor.permute(0, 2, 1, 3).reshape(batch_size * head_size, seq_len, dim // head_size)
        return tensor

    def reshape_batch_dim_to_heads(self, tensor):
        batch_size, seq_len, dim = tensor.shape
        head_size = self.heads
        tensor = tensor.reshape(batch_size // head_size, head_size, seq_len, dim)
        tensor = tensor.permute(0, 2, 1, 3).reshape(batch_size // head_size, seq_len, dim * head_size)
        return tensor

    def set_attention_slice(self, slice_size):
        if slice_size is not None and slice_size > self.sliceable_head_dim:
            raise ValueError(f"slice_size {slice_size} has to be smaller or equal to {self.sliceable_head_dim}.")

        self._slice_size = slice_size

    def forward(self, hidden_states, encoder_hidden_states=None, attention_mask=None):
        batch_size, sequence_length, _ = hidden_states.shape

        encoder_hidden_states = encoder_hidden_states

        if self.group_norm is not None:
            hidden_states = self.group_norm(hidden_states.transpose(1, 2)).transpose(1, 2)

        query = self.to_q(hidden_states)
        dim = query.shape[-1]
        query = self.reshape_heads_to_batch_dim(query)

        if self.added_kv_proj_dim is not None:
            key = self.to_k(hidden_states)
            value = self.to_v(hidden_states)
            encoder_hidden_states_key_proj = self.add_k_proj(encoder_hidden_states)
            encoder_hidden_states_value_proj = self.add_v_proj(encoder_hidden_states)

            key = self.reshape_heads_to_batch_dim(key)
            value = self.reshape_heads_to_batch_dim(value)
            encoder_hidden_states_key_proj = self.reshape_heads_to_batch_dim(encoder_hidden_states_key_proj)
            encoder_hidden_states_value_proj = self.reshape_heads_to_batch_dim(encoder_hidden_states_value_proj)

            key = torch.concat([encoder_hidden_states_key_proj, key], dim=1)
            value = torch.concat([encoder_hidden_states_value_proj, value], dim=1)
        else:
            encoder_hidden_states = encoder_hidden_states if encoder_hidden_states is not None else hidden_states
            key = self.to_k(encoder_hidden_states)
            value = self.to_v(encoder_hidden_states)

            key = self.reshape_heads_to_batch_dim(key)
            value = self.reshape_heads_to_batch_dim(value)

        if attention_mask is not None:
            if attention_mask.shape[-1] != query.shape[1]:
                target_length = query.shape[1]
                attention_mask = F.pad(attention_mask, (0, target_length), value=0.0)
                attention_mask = attention_mask.repeat_interleave(self.heads, dim=0)

        # attention, what we cannot get enough of
        if self._use_memory_efficient_attention_xformers:
            hidden_states = self._memory_efficient_attention_xformers(query, key, value, attention_mask)
            # Some versions of xformers return output in fp32, cast it back to the dtype of the input
            hidden_states = hidden_states.to(query.dtype)
        else:
            if self._slice_size is None or query.shape[0] // self._slice_size == 1:
                hidden_states = self._attention(query, key, value, attention_mask)
            else:
                hidden_states = self._sliced_attention(query, key, value, sequence_length, dim, attention_mask)

        # linear proj
        hidden_states = self.to_out[0](hidden_states)

        # dropout
        hidden_states = self.to_out[1](hidden_states)
        return hidden_states

    def _attention(self, query, key, value, attention_mask=None):
        if self.upcast_attention:
            query = query.float()
            key = key.float()

        attention_scores = torch.baddbmm(
            torch.empty(query.shape[0], query.shape[1], key.shape[1], dtype=query.dtype, device=query.device),
            query,
            key.transpose(-1, -2),
            beta=0,
            alpha=self.scale,
        )

        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask

        if self.upcast_softmax:
            attention_scores = attention_scores.float()

        attention_probs = attention_scores.softmax(dim=-1)

        # cast back to the original dtype
        attention_probs = attention_probs.to(value.dtype)

        # compute attention output
        hidden_states = torch.bmm(attention_probs, value)

        # reshape hidden_states
        hidden_states = self.reshape_batch_dim_to_heads(hidden_states)
        return hidden_states

    def _sliced_attention(self, query, key, value, sequence_length, dim, attention_mask):
        batch_size_attention = query.shape[0]
        hidden_states = torch.zeros(
            (batch_size_attention, sequence_length, dim // self.heads), device=query.device, dtype=query.dtype
        )
        slice_size = self._slice_size if self._slice_size is not None else hidden_states.shape[0]
        for i in range(hidden_states.shape[0] // slice_size):
            start_idx = i * slice_size
            end_idx = (i + 1) * slice_size

            query_slice = query[start_idx:end_idx]
            key_slice = key[start_idx:end_idx]

            if self.upcast_attention:
                query_slice = query_slice.float()
                key_slice = key_slice.float()

            attn_slice = torch.baddbmm(
                torch.empty(slice_size, query.shape[1], key.shape[1], dtype=query_slice.dtype, device=query.device),
                query_slice,
                key_slice.transpose(-1, -2),
                beta=0,
                alpha=self.scale,
            )

            if attention_mask is not None:
                attn_slice = attn_slice + attention_mask[start_idx:end_idx]

            if self.upcast_softmax:
                attn_slice = attn_slice.float()

            attn_slice = attn_slice.softmax(dim=-1)

            # cast back to the original dtype
            attn_slice = attn_slice.to(value.dtype)
            attn_slice = torch.bmm(attn_slice, value[start_idx:end_idx])

            hidden_states[start_idx:end_idx] = attn_slice

        # reshape hidden_states
        hidden_states = self.reshape_batch_dim_to_heads(hidden_states)
        return hidden_states

    def _memory_efficient_attention_xformers(self, query, key, value, attention_mask):
        # TODO attention_mask
        query = query.contiguous()
        key = key.contiguous()
        value = value.contiguous()
        hidden_states = xformers.ops.memory_efficient_attention(query, key, value, attn_bias=attention_mask)
        hidden_states = self.reshape_batch_dim_to_heads(hidden_states)
        return hidden_states


## example

In [11]:
class CrossAttention(nn.Module):
    r"""
    A cross attention layer.

    Parameters:
        query_dim (`int`): The number of channels in the query.
        cross_attention_dim (`int`, *optional*):
            The number of channels in the encoder_hidden_states. If not given, defaults to `query_dim`.
        heads (`int`,  *optional*, defaults to 8): The number of heads to use for multi-head attention.
        dim_head (`int`,  *optional*, defaults to 64): The number of channels in each head.
        dropout (`float`, *optional*, defaults to 0.0): The dropout probability to use.
        bias (`bool`, *optional*, defaults to False):
            Set to `True` for the query, key, and value linear layers to contain a bias parameter.
    """

    def __init__(
        self,
        query_dim: int = 2,
        heads: int = 8,
        dim_head: int = 64,
        bias=False
    ):
        super().__init__()
        inner_dim = dim_head * heads


        self.heads = heads

        self.to_q = nn.Linear(query_dim, inner_dim, bias=bias)
        self.to_k = nn.Linear(query_dim, inner_dim, bias=bias)
        self.to_v = nn.Linear(query_dim, inner_dim, bias=bias)

        self.to_out = nn.ModuleList([])
        self.to_out.append(nn.Linear(inner_dim, query_dim))


    def reshape_heads_to_batch_dim(self, tensor):
        batch_size, seq_len, dim = tensor.shape
        head_size = self.heads
        tensor = tensor.reshape(batch_size, seq_len, head_size, dim // head_size)
        tensor = tensor.permute(0, 2, 1, 3).reshape(batch_size * head_size, seq_len, dim // head_size)
        return tensor

    def reshape_batch_dim_to_heads(self, tensor):
        batch_size, seq_len, dim = tensor.shape
        head_size = self.heads
        tensor = tensor.reshape(batch_size // head_size, head_size, seq_len, dim)
        tensor = tensor.permute(0, 2, 1, 3).reshape(batch_size // head_size, seq_len, dim * head_size)
        return tensor


    def forward(self, hidden_states, encoder_hidden_states=None, attention_mask=None):
        batch_size, sequence_length, _ = hidden_states.shape

        query = self.to_q(hidden_states)
        print("after to_q (linear )", query.shape )
        key = self.to_k(hidden_states)
        value = self.to_v(hidden_states)

        dim = query.shape[-1]
        query = self.reshape_heads_to_batch_dim(query)
        print("after reshape (headstobatch )", query.shape )
        key = self.reshape_heads_to_batch_dim(key)
        value = self.reshape_heads_to_batch_dim(value)

        # attention, what we cannot get enough of
        hidden_states = self._attention(query, key, value)

        # linear proj
        hidden_states = self.to_out[0](hidden_states)
        print("after to_out (linear )", hidden_states.shape )

        return hidden_states

    def _attention(self, query, key, value):

        attention_scores = torch.baddbmm(
            torch.empty(query.shape[0], query.shape[1], key.shape[1], dtype=query.dtype, device=query.device),
            query,
            key.transpose(-1, -2),
            beta=0,
            alpha=1,
        )

        attention_probs = attention_scores.softmax(dim=-1)
        print(" attention weight shape ", attention_probs.shape)
        # cast back to the original dtype
        attention_probs = attention_probs.to(value.dtype)

        # compute attention output
        hidden_states = torch.bmm(attention_probs, value)
        print(" attention before reshapebatchtohead ", hidden_states.shape)

        # reshape hidden_states
        hidden_states = self.reshape_batch_dim_to_heads(hidden_states)
        print(" attention after reshapebatchtohead ", hidden_states.shape)
        return hidden_states

# Initialize the module
cross_attention_layer = CrossAttention(query_dim,heads, dim_head )

In [12]:
batch_size = 2
heads = 2
seq_len = 6
query_dim = 3
dim_head=2



hidden_states = torch.rand(batch_size, seq_len, query_dim)
print("input shape:", hidden_states.shape)


output = cross_attention_layer(hidden_states)
print("output shape:", output.shape)



input shape: torch.Size([2, 6, 3])
after to_q (linear ) torch.Size([2, 6, 4])
after reshape (headstobatch ) torch.Size([4, 6, 2])
 attention weight shape  torch.Size([4, 6, 6])
 attention before reshapebatchtohead  torch.Size([4, 6, 2])
 attention after reshapebatchtohead  torch.Size([2, 6, 4])
after to_out (linear ) torch.Size([2, 6, 3])
output shape: torch.Size([2, 6, 3])


# Import

In [20]:
pip install einops


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00


In [21]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import numpy as np
from typing import Optional
from einops import rearrange

#1.self_attention

##1.1alg

In [2]:
class SelfAttention(nn.Module):

    def __init__(self, d, d_k_q, d_v  ):
        super().__init__()
        self.d_out_kq = d_k_q
        self.W_query = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_key   = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_value = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_v)).float())

        #print("W_query shape:", self.W_query)
        print("W_key shape:", self.W_key, self.W_key.shape)

        #print("W_value shape:", self.W_value)



    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        print("keys ", keys, keys.shape )
        print("x", x, x.shape )
        #print("queries ", queries)
        #print("values ", values)
        attn_scores = queries @ keys.transpose(-2,-1)  # unnormalized attention weights
        #print("attn_scores ", attn_scores )
        attn_weights = torch.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1
        )

        #print("attn_weights ", attn_weights)
        context_vec = attn_weights @ values
        #print("context_vec ", context_vec)
        return context_vec


class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d, d_k_q, d_v, num_heads):
        super().__init__()
        self.num_heads = num_heads

        self.heads = nn.ModuleList(
            [SelfAttention(d, d_k_q, d_v )
             for _ in range(num_heads)]
        )
        print("head ", dir(self.heads[0]))


    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [3]:
# Example usage for MultiHeadAttentionWrapper
d = 3
d_k_q = 2
d_v = 2
num_heads = 2

seq_len = 6
batch_size = 2



multi_head_attention = MultiHeadAttentionWrapper(d, d_k_q, d_v, num_heads)
x = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
print("MultiHeadAttentionWrapper input shape:", x.shape,x)
output = multi_head_attention(x)
#print("MultiHeadAttentionWrapper output shape:", output)
print("MultiHeadAttentionWrapper output shape:" , output.shape )
#output shape [batch_size, seq_len, num_heads * d_v]


W_key shape: Parameter containing:
tensor([[2., 1.],
        [0., 1.],
        [0., 0.]], requires_grad=True) torch.Size([3, 2])
W_key shape: Parameter containing:
tensor([[2., 1.],
        [0., 1.],
        [2., 2.]], requires_grad=True) torch.Size([3, 2])
head  ['T_destination', 'W_key', 'W_query', 'W_value', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get

In [4]:
A=torch.tensor([[1,2,3],[4,5,6]])
print(A)
print(A.shape)
B=torch.tensor([[4,5,6],[7,8,9]])
print(B)
print(B.shape)
print(torch.cat((A,B),dim=-1))

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])
tensor([[4, 5, 6],
        [7, 8, 9]])
torch.Size([2, 3])
tensor([[1, 2, 3, 4, 5, 6],
        [4, 5, 6, 7, 8, 9]])


##1.2two methods compare

In [44]:
# Define the selfttention class from Method 1
class SelfAttention(nn.Module):

    def __init__(self, d, d_k_q, d_v  ):
        super().__init__()
        self.d_out_kq = d_k_q
        self.W_query = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_key   = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_value = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_v)).float())

        #print("W_query shape:", self.W_query)
        #print("W_key shape:", self.W_key, self.W_key.shape)

        #print("W_value shape:", self.W_value)



    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        #print("keys ", keys, keys.shape )
        #print("x", x, x.shape )
        #print("queries ", queries)
        #print("values ", values)
        attn_scores = queries @ keys.transpose(-2,-1)  # unnormalized attention weights
        #print("attn_scores ", attn_scores )
        attn_weights = torch.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1
        )

        #print("attn_weights ", attn_weights)
        context_vec = attn_weights @ values
        #print("context_vec ", context_vec)
        return context_vec


class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d, d_k_q, d_v, num_heads):
        super().__init__()
        self.num_heads = num_heads

        self.heads = nn.ModuleList(
            [SelfAttention(d, d_k_q, d_v )
             for _ in range(num_heads)]
        )
        print("head ", dir(self.heads[0]))

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
##################
# Define the QKVAttention class from Method 2
class QKVAttention(nn.Module):
    def __init__(self, n_heads):
        super().__init__()
        self.heads = n_heads

    def forward(self, qkv):
        bs, width, length = qkv.shape
        print("qkv shape:",  qkv.shape)
        assert width % (3 * self.heads) == 0
        ch = width // (3 * self.heads)
        q, k, v = qkv.chunk(3, dim=1)
        print("q",  q.shape)
        print("k", k.shape)
        print("v", v.shape )
        scale = 1 / math.sqrt(math.sqrt(ch))
        print("q*scale shape ", (q*scale).shape)
        print("new shape ", bs * self.heads, ch, length  )
        weight = torch.einsum(
            "bct,bcs->bts",
            (q * scale).view(bs * self.heads, ch, length),
            (k * scale).view(bs * self.heads, ch, length),
        )  # More stable with f16 than dividing afterwards
        weight = torch.softmax(weight.float(), dim=-1).type(weight.dtype)
        a = torch.einsum(
            "bts,bcs->bct", weight, v.reshape(bs * self.heads, ch, length)
        )
        return a.reshape(bs, -1, length)

# Example parameters
d = 4
d_k_q = 12
d_v = d_k_q
num_heads = 1
seq_len = 6
batch_size = 2
# Example input
x = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
print("MultiHeadAttentionWrapper input shape:",   x.shape)

# Initialize Method 1
multihead_attention_1 = MultiHeadAttentionWrapper(d, d_k_q, d_v, num_heads)
output_1 = multihead_attention_1(x)
#print("MultiHeadAttentionWrapper output shape:", output_1)
print("MultiHeadAttentionWrapper output shape:" , output_1.shape )
print("----------------------------------------------")
# Initialize Method 2
# For Method 2, we need to prepare qkv input
# We'll use the same weights as in Method 1 to ensure identical outputs
queries = [x @ head.W_query for head in multihead_attention_1.heads]
keys = [x @ head.W_key for head in multihead_attention_1.heads]
values = [x @ head.W_value for head in multihead_attention_1.heads]
#print("queries before cat ", queries)

# Concatenate Q, K, V for Method 2 input
queries = torch.cat(queries, dim=-1)
keys = torch.cat(keys, dim=-1)
values = torch.cat(values, dim=-1)
#print("queries after cat", queries)

# Stack the heads to prepare qkv input
print("before permute qkv", torch.cat((queries, keys, values), dim=-1).shape )
qkv = torch.cat((queries, keys, values), dim=-1).permute(0, 2, 1)
print("after permute qkv", qkv.shape  )


# Initialize and run QKVAttention
qkv_attention = QKVAttention(num_heads)
output_2 = qkv_attention(qkv)
#output shape [batch_size, num_heads * d_v,seq_len], here d_out_v = d_k_q

# Permute output_1 to match output_2 shape
output_1 = output_1.permute(0, 2, 1)

# Compare the outputs
print("Output from Method 1:\n", output_1.shape)
print("Output from Method 2:\n",  output_2.shape)

# Check if the outputs are close enough
print("Are the outputs close? ", torch.allclose(output_1, output_2, atol=1e-6))


MultiHeadAttentionWrapper input shape: torch.Size([2, 6, 4])
head  ['T_destination', 'W_key', 'W_query', 'W_value', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', 

##1.3attention_map
https://github.com/google/prompt-to-prompt/blob/main/prompt-to-prompt_stable.ipynb

#2.CrossAttention

##2.1alg

In [48]:
class CrossAttention(nn.Module):

    def __init__(self, d, d_k_q, d_v):
        super().__init__()
        self.d_out_kq = d_k_q
        self.W_query = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_key   = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_value = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_v)).float())

    def forward(self, x_1, x_2):           # x_2 is new
        queries_1 = x_1 @ self.W_query

        keys_2 = x_2 @ self.W_key          # new
        values_2 = x_2 @ self.W_value      # new

        attn_scores = queries_1 @ keys_2.transpose(-2, -1) # new
        attn_weights = torch.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1)

        context_vec = attn_weights @ values_2
        return context_vec

class CrossMultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d, d_k_q, d_v, num_heads):
        super().__init__()
        self.heads = nn.ModuleList(
            [CrossAttention(d, d_k_q, d_v)
             for _ in range(num_heads)]
        )
        print("heads ", dir(self.heads[0]))

    def forward(self, x_1, x_2):
        return torch.cat([head(x_1, x_2) for head in self.heads], dim=-1)



In [51]:
# Example usage for CrossMultiHeadAttentionWrapper
d = 3
d_k_q = 2
d_v = 2
num_heads = 2
batch_size = 1
seq_len = 6

cross_multi_head_attention = CrossMultiHeadAttentionWrapper(d, d_k_q, d_v, num_heads)
x_1 = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
x_2 = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
print("MultiHeadAttentionWrapper input1 shape:", x_1)
print("MultiHeadAttentionWrapper input2 shape:", x_2)
output = cross_multi_head_attention(x_1,x_2)
print("MultiHeadAttentionWrapper output shape:", output)
print("MultiHeadAttentionWrapper output shape:" , output.shape )
#output shape [batch_size, seq_len, num_heads * d_v]



heads  ['T_destination', 'W_key', 'W_query', 'W_value', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameter

##2.2 two methods compare

In [60]:
# Define the QKVAttention class from Method 1
class CrossAttention(nn.Module):

    def __init__(self, d, d_k_q, d_v):
        super().__init__()
        self.d_out_kq = d_k_q
        self.W_query = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_key   = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_value = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_v)).float())

    def forward(self, x_1, x_2):           # x_2 is new
        queries_1 = x_1 @ self.W_query

        keys_2 = x_2 @ self.W_key          # new
        values_2 = x_2 @ self.W_value      # new
        print("queries_1 shape:", queries_1)
        print("keys_2 shape:", keys_2)
        print("values_2 shape:", values_2)
        attn_scores = queries_1 @ keys_2.transpose(-2, -1) # new
        attn_weights = torch.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1)

        context_vec = attn_weights @ values_2
        return context_vec

    def reshape_heads_to_batch_dim(self, tensor):
        batch_size, seq_len, dim = tensor.shape
        head_size = self.n_heads
        tensor = tensor.reshape(batch_size, seq_len, head_size, dim // head_size)
        tensor = tensor.permute(0, 2, 1, 3).reshape(batch_size * head_size, seq_len, dim // head_size)
        return tensor

    def reshape_batch_dim_to_heads(self, tensor):
        batch_size, seq_len, dim = tensor.shape
        head_size = self.heads
        tensor = tensor.reshape(batch_size // head_size, head_size, seq_len, dim)
        tensor = tensor.permute(0, 2, 1, 3).reshape(batch_size // head_size, seq_len, dim * head_size)
        return tensor

class CrossMultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d, d_k_q, d_v, num_heads):
        super().__init__()
        self.heads = nn.ModuleList(
            [CrossAttention(d, d_k_q, d_v)
             for _ in range(num_heads)]
        )
        print("heads ", dir(self.heads[0]))

    def forward(self, x_1, x_2):
        return torch.cat([head(x_1, x_2) for head in self.heads], dim=-1)



##################
# Define the QKVAttention class from Method 2
class CrossQKVAttention(nn.Module):
    def __init__(self, n_heads):
        super().__init__()
        self.n_heads = n_heads

    def forward(self, qkv):
        bs, width, length = qkv.shape
        assert width % (3 * self.n_heads) == 0
        ch = width // (3 * self.n_heads)
        q, k, v = qkv.chunk(3, dim=1)
        print("q", q, q.shape)
        print("k", k.shape)
        print("v", v.shape )
        scale = 1 / math.sqrt(math.sqrt(ch))

        weight = torch.einsum(
            "bct,bcs->bts",
            (q * scale).view(bs * self.n_heads, ch, length),
            (k * scale).view(bs * self.n_heads, ch, length),
        )  # More stable with f16 than dividing afterwards
        weight = torch.softmax(weight.float(), dim=-1).type(weight.dtype)
        a = torch.einsum(
            "bts,bcs->bct", weight, v.reshape(bs * self.n_heads, ch, length)
        )
        return a.reshape(bs, -1, length)

# Example parameters
# Example parameters
d = 4
d_k_q = 2
d_v = 2
num_heads = 2
seq_len = 6
batch_size = 2

# Example input
x_1 = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
x_2 = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
print("CrossMultiHeadAttentionWrapper input 1 shape:", x_1)
print("CrossMultiHeadAttentionWrapper input 2 shape:", x_2)

# Initialize Method 1
cross_multi_head_attention = CrossMultiHeadAttentionWrapper(d, d_k_q, d_v, num_heads)
output_1 = cross_multi_head_attention(x_1,x_2)
print("MultiHeadAttentionWrapper output shape:", output_1)
print("MultiHeadAttentionWrapper output shape:" , output_1.shape )

print("-----------------------------------------------------")
# Initialize Method 2
# For Method 2, we need to prepare qkv input
# We'll use the same weights as in Method 1 to ensure identical outputs
queries = [x_1 @ head.W_query for head in cross_multi_head_attention.heads]
keys = [x_2 @ head.W_key for head in cross_multi_head_attention.heads]
values = [x_2 @ head.W_value for head in cross_multi_head_attention.heads]
print("queries before cat ", queries)
print("keys before cat ", keys)
print("values before cat ", values)


# Concatenate Q, K, V for Method 2 input
queries = torch.cat(queries, dim=-1)
keys = torch.cat(keys, dim=-1)
values = torch.cat(values, dim=-1)
print("queries after cat", queries)

# Stack the heads to prepare qkv input
print("before permute qkv", torch.cat((queries, keys, values), dim=-1) )
qkv = torch.cat((queries, keys, values), dim=-1).permute(0, 2, 1)
print("after permute qkv", qkv  )

# Initialize and run QKVAttention
cross_qkv_attention = CrossQKVAttention(num_heads)
output_2 = cross_qkv_attention(qkv)
#output shape [batch_size, num_heads * d_v,seq_len], here d_out_v = d_k_q

# Permute output_1 to match output_2 shape
output_1 = output_1.permute(0, 2, 1)

# Compare the outputs
print("Output from Method 1:\n", output_1)
print("Output from Method 2:\n", output_2)

# Check if the outputs are close enough
print("Are the outputs close? ", torch.allclose(output_1, output_2, atol=1e-6))


CrossMultiHeadAttentionWrapper input 1 shape: tensor([[[1., 2., 3., 0.],
         [1., 2., 2., 0.],
         [2., 2., 3., 1.],
         [1., 2., 2., 2.],
         [3., 1., 3., 0.],
         [0., 3., 3., 0.]],

        [[1., 3., 3., 3.],
         [2., 0., 2., 3.],
         [1., 2., 0., 3.],
         [3., 2., 2., 3.],
         [1., 3., 3., 0.],
         [1., 2., 0., 1.]]])
CrossMultiHeadAttentionWrapper input 2 shape: tensor([[[0., 3., 1., 3.],
         [3., 2., 0., 2.],
         [2., 2., 0., 0.],
         [1., 3., 0., 2.],
         [3., 0., 1., 3.],
         [0., 1., 1., 2.]],

        [[3., 0., 1., 2.],
         [0., 0., 3., 0.],
         [2., 2., 2., 0.],
         [3., 3., 2., 1.],
         [2., 3., 3., 3.],
         [1., 1., 3., 1.]]])
heads  ['T_destination', 'W_key', 'W_query', 'W_value', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash_

# test AnimateDiff versatileAttention

In [34]:
d=2
batch_size=5
seq_len=6
tensor=torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
print(tensor.shape)

output1=rearrange(tensor, "(b f) d c -> (b d) f c", f=5)
print(output.shape)

torch.Size([5, 6, 2])
torch.Size([6, 5, 2])


In [35]:
class PositionalEncoding(nn.Module):
    def __init__(
        self,
        d_model,
        dropout = 0.,
        max_len = 24
    ):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

pos_encoder = PositionalEncoding(
            d_model=2,#kwargs["query_dim"],
            dropout=0.,
            max_len=24,
        )


In [36]:
output2=pos_encoder(output1)
print(output2.shape)

torch.Size([6, 5, 2])


#test JoliGEN UNet_QKVattention

In [48]:
channels=512
d=3
qkv = nn.Conv1d(channels, channels * 3, 1)
input1=torch.randint(low=0,high=d,size=(5, 512,16*16 )).float()
print(input1.shape)
op1=qkv(input1)
print(op1.shape)



torch.Size([5, 512, 256])
torch.Size([5, 1536, 256])


In [1]:
import torch
import torch.nn as nn

# Define the zero_module function
def zero_module(module):
    """
    Zero out the parameters of a module and return it.
    """
    for p in module.parameters():
        p.detach().zero_()
    return module

# Define a neural network module
class MyNetwork(nn.Module):
    def __init__(self, channels):
        super(MyNetwork, self).__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = zero_module(nn.Conv1d(channels, channels, kernel_size=1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x

# Create an instance of the network
channels = 16
net = MyNetwork(channels=channels)

# Print the network to verify the layers
print(net)

# Create a dummy input tensor
input_tensor = torch.randn(1, channels, 10)  # (batch_size, channels, length)

# Forward pass
output_tensor = net(input_tensor)

# Print the output
print(output_tensor)


MyNetwork(
  (conv1): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
)
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<ConvolutionBackward0>)


In [9]:
import torch
import math
class QKVAttention(nn.Module):
    """
    A module which performs QKV attention and splits in a different order.
    """

    def __init__(self, n_heads):
        super().__init__()
        self.n_heads = n_heads
    def forward(self, qkv):
        """
        Apply QKV attention.
        :param qkv: an [N x (3 * H * C) x T] tensor of Qs, Ks, and Vs.
        :return: an [N x (H * C) x T] tensor after attention.
        """
        bs, width, length = qkv.shape
        assert width % (3 * self.n_heads) == 0
        ch = width // (3 * self.n_heads)
        q, k, v = qkv.chunk(3, dim=1)
        print("q", q.shape)
        print("k", k.shape)
        print("v", v.shape )
        scale = 1 / math.sqrt(math.sqrt(ch))
        weight = torch.einsum(
            "bct,bcs->bts",
            (q * scale).view(bs * self.n_heads, ch, length),
            (k * scale).view(bs * self.n_heads, ch, length),
        )  # More stable with f16 than dividing afterwards
        print("weight", weight.shape)
        weight = torch.softmax(weight.float(), dim=-1).type(weight.dtype)
        a = torch.einsum(
            "bts,bcs->bct", weight, v.reshape(bs * self.n_heads, ch, length)
        )
        print("a shape ", a.shape)
        print( a.reshape(bs, -1, length).shape )
        return a.reshape(bs, -1, length)



# Example usage
qkv = torch.randint(low=0, high=10, size=(1, 192, 256)).float()
n_heads = 2

att_module = QKVAttention(n_heads)
output = att_module(qkv)
print("Output shape:", output.shape)  #

q torch.Size([1, 64, 256])
k torch.Size([1, 64, 256])
v torch.Size([1, 64, 256])
weight torch.Size([2, 256, 256])
a shape  torch.Size([2, 32, 256])
torch.Size([1, 64, 256])
Output shape: torch.Size([1, 64, 256])
